# [Module 1.7] DDP 훈련

본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 아래는 세이지메이커의 어떤 피쳐도 사용하지 않고, PyTorch 만을 사용해서 훈련 합니다.

# PyTorch CIFAR-10 local training  



In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-cnn-cifar10"

role = sagemaker.get_execution_role()

### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [45]:
s3_inputs = sagemaker_session.upload_data(path="../data", bucket=bucket, key_prefix="data/cifar10")
print("s3 inputs: ", s3_inputs)

s3 inputs:  s3://sagemaker-us-east-1-057716757052/data/cifar10


# Construct a script for training 
- epoch 10 , 20
    - 각각 테스트 정확도 55.2, 62.9


In [69]:
import os
import subprocess

 #instance_type = "local_gpu"
instance_type="ml.p3.16xlarge"

if instance_type == 'local_gpu':
    sess = sagemaker.local.LocalSession()
    inputs = 'file://../data'    
    instance_count = 1
    hyperparameters={"epochs": 1, 
                     'batch-size': 128,                     
                     'lr': 0.01,
                    }    
    print(instance_type)
else:
    sess = sagemaker.Session()
    inputs = s3_inputs
    instance_count = 2
    hyperparameters={"epochs": 20, 
                     'batch-size': 128,                     
                     'lr': 0.01,
                    }        
    print(instance_type)



ml.p3.16xlarge


In [70]:
from sagemaker.pytorch import PyTorch

job_name ='cifar10-ddp'

estimator = PyTorch(
    entry_point="train_ddp.py",    
    source_dir='source',    
    base_job_name = job_name,
    role=role,
    framework_version="1.8.1",
    py_version="py36",
    # For training with multinode distributed training, set this count. Example: 2
    instance_count= instance_count,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type= instance_type,
    sagemaker_session= sess,
    # Training using SMDataParallel Distributed Training Framework
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
    hyperparameters=hyperparameters,    
    debugger_hook_config=False,
)
estimator.fit({"training" : inputs})
#estimator.fit({"training" : "file://../data"})

2021-07-09 08:48:04 Starting - Starting the training job...
2021-07-09 08:48:29 Starting - Launching requested ML instancesProfilerReport-1625820484: InProgress
.........
2021-07-09 08:49:49 Starting - Preparing the instances for training.........
2021-07-09 08:51:30 Downloading - Downloading input data......
2021-07-09 08:52:35 Training - Downloading the training image...............
2021-07-09 08:55:05 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-07-09 08:55:05,445 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-07-09 08:55:05,523 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-07-09 08:55:06,034 sagemaker-training-toolkit INFO     Imported fra

In [65]:
ddp_artifact_path = estimator.model_data
print("ddp_artifact_path: ", ddp_artifact_path)


%store ddp_artifact_path

ddp_artifact_path:  s3://sagemaker-us-east-1-057716757052/cifar10-ddp-2021-07-09-08-40-36-084/model.tar.gz
Stored 'ddp_artifact_path' (str)


In [55]:
! aws s3 ls {ddp_artifact_path} --recursive

2021-07-09 07:44:46     230809 cifar10-ddp-2021-07-09-07-36-44-829/output/model.tar.gz
